In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time 
from torch import utils
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torchtext
from tqdm import tqdm
from torchtext.datasets import IMDB

from torchtext.datasets.imdb import NUM_LINES
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

import os
import sys
import logging
import logging
logging.basicConfig(
    level=logging.WARN, stream=sys.stdout, \
    format="%(asctime)s (%(module)s:%(lineno)d) %(levelname)s: %(message)s")

VOCAB_SIZE = 15000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.warning(f"device is {device}")
VOCAB_SIZE = 15000 
BATCH_SIZE = 64

def yeild_tokens(train_data_iter, tokenizer):
    for i, sample in enumerate(train_data_iter):
        label, comment = sample
        yield tokenizer(comment)
        
    
class GCNN(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, embedding_dim=64, num_class=2):
        super(GCNN, self).__init__()
        
        self.embedding_table = nn.Embedding(vocab_size, embedding_dim).to(device)
        nn.init.xavier_uniform_(self.embedding_table.weight).to(device)
        
        self.conv_A_1 = nn.Conv1d(embedding_dim, 64, 15, stride=7).to(device)
        self.conv_B_1 = nn.Conv1d(embedding_dim, 64, 15, stride=7).to(device)
        
        self.conv_A_2 = nn.Conv1d(64, 64, 15, stride=7).to(device)
        self.conv_B_2 = nn.Conv1d(64, 64, 15, stride=7).to(device)
        
        self.output_linear1 = nn.Linear(64, 128).to(device)
        self.output_linear2 = nn.Linear(128, num_class).to(device)
        
    def forward(self, word_index):
        word_embedding = self.embedding_table(word_index)  
        
        word_embedding = word_embedding.transpose(1, 2)  
        A = self.conv_A_1(word_embedding)
        B = self.conv_B_1(word_embedding)
        H = A * torch.sigmoid(B) 
        
        A = self.conv_A_2(H)
        B = self.conv_B_2(H)
        H = A * torch.sigmoid(B) 
        
        pool_output = torch.mean(H, dim=-1)  
        linear1_output = self.output_linear1(pool_output)
        
        logits = self.output_linear2(linear1_output)
        
        return logits

2023-05-06 07:11:13,077 (2091545762:27) WARNING: device is cuda


In [2]:
train_data_iter = IMDB(root="data", split="train") 
tokenizer = get_tokenizer("basic_english")

vocab = build_vocab_from_iterator(yeild_tokens(train_data_iter, tokenizer), min_freq=20, specials=["<unk>"])
vocab.set_default_index(0)  
print(f'Vocal size: {len(vocab)}')

Vocal size: 13351


In [3]:
def collate_fn(batch):
    target = []
    token_index = []
    max_length = 0  
    for i, (label, comment) in enumerate(batch):
        tokens = tokenizer(comment)
        token_index.append(vocab(tokens)) 
        
        if len(tokens) > max_length:
            max_length = len(tokens)
        
        if label == "pos":
            target.append(0)
        else:
            target.append(1)

    token_index = [index + [0]*(max_length-len(index)) for index in token_index]
    return (torch.tensor(target).to(torch.int64), torch.tensor(token_index).to(torch.int32))

In [4]:
def train(train_data_loader, eval_data_loader, model, optimizer, num_epoch, log_step_interval, save_step_interval, \
          eval_step_interval, save_path, resume=""):
    start_epoch = 0
    start_step = 0
    if resume != "":
        logging.warning(f"loading from {resume}")
        checkpoint = torch.load(resume)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        start_step = checkpoint['step']
        
    for epoch_index in tqdm(range(start_epoch, num_epoch), desc="epoch"):
        ema_loss = 0
        num_batches = len(train_data_loader)
        start_time = time.time()
        
        for batch_index, (target, token_index) in enumerate(train_data_loader):
            target = target.to(device)
            token_index = token_index.to(device)
            optimizer.zero_grad()
            step = num_batches*(epoch_index) + batch_index + 1
            logits = model(token_index)

            bce_loss = F.binary_cross_entropy(torch.sigmoid(logits), F.one_hot(target, num_classes=2).to(torch.float32))
            ema_loss = 0.9 * ema_loss + 0.1 * bce_loss  
            bce_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.1)  
            optimizer.step()   
            
            if step % log_step_interval == 0:
                logging.warning(f"epoch_index: {epoch_index}, batch_index: {batch_index}, ema_loss: {ema_loss}")
                
            if step % save_step_interval == 0:
                os.makedirs(save_path, exist_ok=True)
                save_file = os.path.join(save_path, f"step_{step}.pt")
                torch.save({
                    "epoch": epoch_index,
                    "step": step,
                    "model_state_dict": model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': bce_loss,
                }, save_file)
                logging.warning(f"checkpoint has been saved in {save_file}")
            
            if step % eval_step_interval == 0:
                logging.warning("start to do evaluation...")
                model.eval()
                ema_eval_loss = 0
                total_acc_account = 0
                total_account = 0
                for eval_batch_index, (eval_target, eval_token_index) in enumerate(eval_data_loader):
                    eval_target = eval_target.to(device)
                    eval_token_index = eval_token_index.to(device)
                    total_account += eval_target.shape[0]
                    eval_logits = model(eval_token_index)
                    total_acc_account += (torch.argmax(eval_logits, dim=-1) == eval_target).sum().item()
                    eval_bce_loss = F.binary_cross_entropy(torch.sigmoid(eval_logits), F.one_hot(eval_target, num_classes=2).to(torch.float32))
                    ema_eval_loss = 0.9 * ema_eval_loss + 0.1 * eval_bce_loss
                logging.warning(f"ema_eval_loss: {ema_eval_loss}, eval_acc: {total_acc_account / total_account}")
                model.train()
        time_period = time.time()-start_time
        logging.warning(f"time cost is: {time_period}")

In [5]:
model = GCNN().to(device)
print("Model total parameters:", sum(p.numel() for p in model.parameters()))
resume = ""

Model total parameters: 1214594


In [6]:
train_data_iter = IMDB(root="data", split="train") 
train_data_loader = torch.utils.data.DataLoader(
    to_map_style_dataset(train_data_iter), batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

eval_data_iter = IMDB(root="data", split="test")  
eval_data_loader = utils.data.DataLoader(
    to_map_style_dataset(eval_data_iter), batch_size=8, collate_fn=collate_fn)

In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
train(train_data_loader, eval_data_loader, model, optimizer, num_epoch=10, log_step_interval=20, \
      save_step_interval=500, eval_step_interval=300, save_path="./gcnn_admw", resume=resume)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

2023-05-06 07:11:45,703 (2306368053:32) WARNING: epoch_index: 0, batch_index: 19, ema_loss: 0.6093685626983643
2023-05-06 07:11:46,065 (2306368053:32) WARNING: epoch_index: 0, batch_index: 39, ema_loss: 0.6835482716560364
2023-05-06 07:11:46,440 (2306368053:32) WARNING: epoch_index: 0, batch_index: 59, ema_loss: 0.6919752359390259
2023-05-06 07:11:46,815 (2306368053:32) WARNING: epoch_index: 0, batch_index: 79, ema_loss: 0.6931185126304626
2023-05-06 07:11:47,190 (2306368053:32) WARNING: epoch_index: 0, batch_index: 99, ema_loss: 0.6931490302085876
2023-05-06 07:11:47,545 (2306368053:32) WARNING: epoch_index: 0, batch_index: 119, ema_loss: 0.6930544376373291
2023-05-06 07:11:47,927 (2306368053:32) WARNING: epoch_index: 0, batch_index: 139, ema_loss: 0.6932767629623413
2023-05-06 07:11:48,298 (2306368053:32) WARNING: epoch_index: 0, batch_index: 159, ema_loss: 0.6930815577507019
2023-05-06 07:11:48,666 (2306368053:32) WARNING: epoch_index: 0, batch_index: 179, ema_loss: 0.69354254007339

epoch:  10%|█         | 1/10 [00:14<02:12, 14.72s/it]

2023-05-06 07:12:00,213 (2306368053:32) WARNING: epoch_index: 1, batch_index: 8, ema_loss: 0.39859920740127563
2023-05-06 07:12:00,603 (2306368053:32) WARNING: epoch_index: 1, batch_index: 28, ema_loss: 0.6076792478561401
2023-05-06 07:12:00,964 (2306368053:32) WARNING: epoch_index: 1, batch_index: 48, ema_loss: 0.6104103326797485
2023-05-06 07:12:01,341 (2306368053:32) WARNING: epoch_index: 1, batch_index: 68, ema_loss: 0.6003459095954895
2023-05-06 07:12:01,721 (2306368053:32) WARNING: epoch_index: 1, batch_index: 88, ema_loss: 0.5812162160873413
2023-05-06 07:12:02,093 (2306368053:32) WARNING: epoch_index: 1, batch_index: 108, ema_loss: 0.5568131804466248
2023-05-06 07:12:02,124 (2306368053:44) WARNING: checkpoint has been saved in ./lstm_admw/step_500.pt
2023-05-06 07:12:02,501 (2306368053:32) WARNING: epoch_index: 1, batch_index: 128, ema_loss: 0.5503867864608765
2023-05-06 07:12:02,884 (2306368053:32) WARNING: epoch_index: 1, batch_index: 148, ema_loss: 0.5341698527336121
2023-05

epoch:  20%|██        | 2/10 [00:29<01:56, 14.56s/it]

...

epoch:  80%|████████  | 8/10 [02:09<00:33, 16.55s/it]

2023-05-06 07:13:55,444 (2306368053:32) WARNING: epoch_index: 8, batch_index: 11, ema_loss: 0.14532990753650665
2023-05-06 07:13:55,825 (2306368053:32) WARNING: epoch_index: 8, batch_index: 31, ema_loss: 0.17528501152992249
2023-05-06 07:13:56,212 (2306368053:32) WARNING: epoch_index: 8, batch_index: 51, ema_loss: 0.20065298676490784
2023-05-06 07:13:56,588 (2306368053:32) WARNING: epoch_index: 8, batch_index: 71, ema_loss: 0.2040051966905594
2023-05-06 07:13:56,966 (2306368053:32) WARNING: epoch_index: 8, batch_index: 91, ema_loss: 0.20196355879306793
2023-05-06 07:13:57,356 (2306368053:32) WARNING: epoch_index: 8, batch_index: 111, ema_loss: 0.22524204850196838
2023-05-06 07:13:57,732 (2306368053:32) WARNING: epoch_index: 8, batch_index: 131, ema_loss: 0.19899876415729523
2023-05-06 07:13:58,116 (2306368053:32) WARNING: epoch_index: 8, batch_index: 151, ema_loss: 0.2154841423034668
2023-05-06 07:13:58,475 (2306368053:32) WARNING: epoch_index: 8, batch_index: 171, ema_loss: 0.20089808

epoch:  90%|█████████ | 9/10 [02:24<00:15, 15.89s/it]

2023-05-06 07:14:09,681 (2306368053:32) WARNING: epoch_index: 9, batch_index: 0, ema_loss: 0.020937254652380943
2023-05-06 07:14:10,048 (2306368053:32) WARNING: epoch_index: 9, batch_index: 20, ema_loss: 0.1571529507637024
2023-05-06 07:14:10,421 (2306368053:32) WARNING: epoch_index: 9, batch_index: 40, ema_loss: 0.21426482498645782
2023-05-06 07:14:10,805 (2306368053:32) WARNING: epoch_index: 9, batch_index: 60, ema_loss: 0.19307458400726318
2023-05-06 07:14:11,199 (2306368053:32) WARNING: epoch_index: 9, batch_index: 80, ema_loss: 0.18596886098384857
2023-05-06 07:14:11,200 (2306368053:47) WARNING: start to do evaluation...
2023-05-06 07:14:18,333 (2306368053:60) WARNING: ema_eval_loss: 0.3897392451763153, eval_acc: 0.88324
2023-05-06 07:14:18,731 (2306368053:32) WARNING: epoch_index: 9, batch_index: 100, ema_loss: 0.1673744171857834
2023-05-06 07:14:19,102 (2306368053:32) WARNING: epoch_index: 9, batch_index: 120, ema_loss: 0.1648043692111969
2023-05-06 07:14:19,489 (2306368053:32) 

epoch: 100%|██████████| 10/10 [02:45<00:00, 16.59s/it]


In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
train(train_data_loader, eval_data_loader, model, optimizer, num_epoch=10, log_step_interval=20, \
      save_step_interval=500, eval_step_interval=300, save_path="./gcnn_sgd", resume=resume)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

2023-05-06 07:14:31,733 (2306368053:32) WARNING: epoch_index: 0, batch_index: 19, ema_loss: 0.1554066389799118
2023-05-06 07:14:32,103 (2306368053:32) WARNING: epoch_index: 0, batch_index: 39, ema_loss: 0.1625068336725235
2023-05-06 07:14:32,468 (2306368053:32) WARNING: epoch_index: 0, batch_index: 59, ema_loss: 0.15402330458164215
2023-05-06 07:14:32,834 (2306368053:32) WARNING: epoch_index: 0, batch_index: 79, ema_loss: 0.146818608045578
2023-05-06 07:14:33,203 (2306368053:32) WARNING: epoch_index: 0, batch_index: 99, ema_loss: 0.1559552401304245
2023-05-06 07:14:33,584 (2306368053:32) WARNING: epoch_index: 0, batch_index: 119, ema_loss: 0.16689442098140717
2023-05-06 07:14:33,955 (2306368053:32) WARNING: epoch_index: 0, batch_index: 139, ema_loss: 0.15291233360767365
2023-05-06 07:14:34,318 (2306368053:32) WARNING: epoch_index: 0, batch_index: 159, ema_loss: 0.16074633598327637
2023-05-06 07:14:34,679 (2306368053:32) WARNING: epoch_index: 0, batch_index: 179, ema_loss: 0.14598271250

epoch:  10%|█         | 1/10 [00:14<02:07, 14.12s/it]

2023-05-06 07:14:45,667 (2306368053:32) WARNING: epoch_index: 1, batch_index: 8, ema_loss: 0.09246909618377686
2023-05-06 07:14:46,018 (2306368053:32) WARNING: epoch_index: 1, batch_index: 28, ema_loss: 0.15972937643527985
2023-05-06 07:14:46,361 (2306368053:32) WARNING: epoch_index: 1, batch_index: 48, ema_loss: 0.14995746314525604
2023-05-06 07:14:46,711 (2306368053:32) WARNING: epoch_index: 1, batch_index: 68, ema_loss: 0.16575682163238525
2023-05-06 07:14:47,065 (2306368053:32) WARNING: epoch_index: 1, batch_index: 88, ema_loss: 0.15892362594604492
2023-05-06 07:14:47,433 (2306368053:32) WARNING: epoch_index: 1, batch_index: 108, ema_loss: 0.1523325890302658
2023-05-06 07:14:47,447 (2306368053:44) WARNING: checkpoint has been saved in ./lstm_sgd/step_500.pt
2023-05-06 07:14:47,806 (2306368053:32) WARNING: epoch_index: 1, batch_index: 128, ema_loss: 0.15043729543685913
2023-05-06 07:14:48,179 (2306368053:32) WARNING: epoch_index: 1, batch_index: 148, ema_loss: 0.14687307178974152
20

epoch:  20%|██        | 2/10 [00:28<01:52, 14.09s/it]

...

epoch:  80%|████████  | 8/10 [02:06<00:32, 16.22s/it]

2023-05-06 07:16:38,538 (2306368053:32) WARNING: epoch_index: 8, batch_index: 11, ema_loss: 0.10781192779541016
2023-05-06 07:16:38,879 (2306368053:32) WARNING: epoch_index: 8, batch_index: 31, ema_loss: 0.18638205528259277
2023-05-06 07:16:39,229 (2306368053:32) WARNING: epoch_index: 8, batch_index: 51, ema_loss: 0.16564466059207916
2023-05-06 07:16:39,593 (2306368053:32) WARNING: epoch_index: 8, batch_index: 71, ema_loss: 0.1617174744606018
2023-05-06 07:16:39,949 (2306368053:32) WARNING: epoch_index: 8, batch_index: 91, ema_loss: 0.18641310930252075
2023-05-06 07:16:40,311 (2306368053:32) WARNING: epoch_index: 8, batch_index: 111, ema_loss: 0.17304843664169312
2023-05-06 07:16:40,672 (2306368053:32) WARNING: epoch_index: 8, batch_index: 131, ema_loss: 0.16761742532253265
2023-05-06 07:16:41,034 (2306368053:32) WARNING: epoch_index: 8, batch_index: 151, ema_loss: 0.1760820597410202
2023-05-06 07:16:41,370 (2306368053:32) WARNING: epoch_index: 8, batch_index: 171, ema_loss: 0.14588132

epoch:  90%|█████████ | 9/10 [02:21<00:15, 15.55s/it]

2023-05-06 07:16:52,440 (2306368053:32) WARNING: epoch_index: 9, batch_index: 0, ema_loss: 0.015034327283501625
2023-05-06 07:16:52,797 (2306368053:32) WARNING: epoch_index: 9, batch_index: 20, ema_loss: 0.12745629251003265
2023-05-06 07:16:53,140 (2306368053:32) WARNING: epoch_index: 9, batch_index: 40, ema_loss: 0.1528920978307724
2023-05-06 07:16:53,519 (2306368053:32) WARNING: epoch_index: 9, batch_index: 60, ema_loss: 0.15322671830654144
2023-05-06 07:16:53,890 (2306368053:32) WARNING: epoch_index: 9, batch_index: 80, ema_loss: 0.15438930690288544
2023-05-06 07:16:53,890 (2306368053:47) WARNING: start to do evaluation...
2023-05-06 07:17:00,981 (2306368053:60) WARNING: ema_eval_loss: 0.6074018478393555, eval_acc: 0.8854
2023-05-06 07:17:01,353 (2306368053:32) WARNING: epoch_index: 9, batch_index: 100, ema_loss: 0.16673730313777924
2023-05-06 07:17:01,719 (2306368053:32) WARNING: epoch_index: 9, batch_index: 120, ema_loss: 0.17227593064308167
2023-05-06 07:17:02,070 (2306368053:32)

epoch: 100%|██████████| 10/10 [02:42<00:00, 16.22s/it]


In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train(train_data_loader, eval_data_loader, model, optimizer, num_epoch=10, log_step_interval=20, \
      save_step_interval=500, eval_step_interval=300, save_path="./gcnn_adm", resume=resume)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

2023-05-06 07:03:43,515 (2306368053:32) WARNING: epoch_index: 0, batch_index: 19, ema_loss: 0.6097363233566284
2023-05-06 07:03:43,904 (2306368053:32) WARNING: epoch_index: 0, batch_index: 39, ema_loss: 0.6846286654472351
2023-05-06 07:03:44,284 (2306368053:32) WARNING: epoch_index: 0, batch_index: 59, ema_loss: 0.691966712474823
2023-05-06 07:03:44,675 (2306368053:32) WARNING: epoch_index: 0, batch_index: 79, ema_loss: 0.6931794881820679
2023-05-06 07:03:45,052 (2306368053:32) WARNING: epoch_index: 0, batch_index: 99, ema_loss: 0.693241536617279
2023-05-06 07:03:45,420 (2306368053:32) WARNING: epoch_index: 0, batch_index: 119, ema_loss: 0.6932549476623535
2023-05-06 07:03:45,779 (2306368053:32) WARNING: epoch_index: 0, batch_index: 139, ema_loss: 0.6930125951766968
2023-05-06 07:03:46,146 (2306368053:32) WARNING: epoch_index: 0, batch_index: 159, ema_loss: 0.6923025846481323
2023-05-06 07:03:46,510 (2306368053:32) WARNING: epoch_index: 0, batch_index: 179, ema_loss: 0.6928043365478516

epoch:  10%|█         | 1/10 [00:14<02:13, 14.78s/it]

2023-05-06 07:03:58,053 (2306368053:32) WARNING: epoch_index: 1, batch_index: 8, ema_loss: 0.3897872567176819
2023-05-06 07:03:58,429 (2306368053:32) WARNING: epoch_index: 1, batch_index: 28, ema_loss: 0.5894634127616882
2023-05-06 07:03:58,792 (2306368053:32) WARNING: epoch_index: 1, batch_index: 48, ema_loss: 0.5994618535041809
2023-05-06 07:03:59,170 (2306368053:32) WARNING: epoch_index: 1, batch_index: 68, ema_loss: 0.593417227268219
2023-05-06 07:03:59,546 (2306368053:32) WARNING: epoch_index: 1, batch_index: 88, ema_loss: 0.5999985337257385
2023-05-06 07:03:59,914 (2306368053:32) WARNING: epoch_index: 1, batch_index: 108, ema_loss: 0.5842818021774292
2023-05-06 07:03:59,944 (2306368053:44) WARNING: checkpoint has been saved in ./lstm_adm/step_500.pt
2023-05-06 07:04:00,333 (2306368053:32) WARNING: epoch_index: 1, batch_index: 128, ema_loss: 0.5790026783943176
2023-05-06 07:04:00,694 (2306368053:32) WARNING: epoch_index: 1, batch_index: 148, ema_loss: 0.5447433590888977
2023-05-06

epoch:  20%|██        | 2/10 [00:29<01:56, 14.57s/it]

...

epoch:  80%|████████  | 8/10 [02:10<00:33, 16.61s/it]

2023-05-06 07:05:53,437 (2306368053:32) WARNING: epoch_index: 8, batch_index: 11, ema_loss: 0.18480123579502106
2023-05-06 07:05:53,800 (2306368053:32) WARNING: epoch_index: 8, batch_index: 31, ema_loss: 0.1791687160730362
2023-05-06 07:05:54,184 (2306368053:32) WARNING: epoch_index: 8, batch_index: 51, ema_loss: 0.20766951143741608
2023-05-06 07:05:54,562 (2306368053:32) WARNING: epoch_index: 8, batch_index: 71, ema_loss: 0.20624375343322754
2023-05-06 07:05:54,941 (2306368053:32) WARNING: epoch_index: 8, batch_index: 91, ema_loss: 0.2022840678691864
2023-05-06 07:05:55,311 (2306368053:32) WARNING: epoch_index: 8, batch_index: 111, ema_loss: 0.20866413414478302
2023-05-06 07:05:55,699 (2306368053:32) WARNING: epoch_index: 8, batch_index: 131, ema_loss: 0.21903684735298157
2023-05-06 07:05:56,083 (2306368053:32) WARNING: epoch_index: 8, batch_index: 151, ema_loss: 0.20894291996955872
2023-05-06 07:05:56,467 (2306368053:32) WARNING: epoch_index: 8, batch_index: 171, ema_loss: 0.20152048

epoch:  90%|█████████ | 9/10 [02:24<00:15, 15.95s/it]

2023-05-06 07:06:07,727 (2306368053:32) WARNING: epoch_index: 9, batch_index: 0, ema_loss: 0.023029809817671776
2023-05-06 07:06:08,093 (2306368053:32) WARNING: epoch_index: 9, batch_index: 20, ema_loss: 0.19749443233013153
2023-05-06 07:06:08,464 (2306368053:32) WARNING: epoch_index: 9, batch_index: 40, ema_loss: 0.18059171736240387
2023-05-06 07:06:08,844 (2306368053:32) WARNING: epoch_index: 9, batch_index: 60, ema_loss: 0.19749075174331665
2023-05-06 07:06:09,225 (2306368053:32) WARNING: epoch_index: 9, batch_index: 80, ema_loss: 0.22881637513637543
2023-05-06 07:06:09,225 (2306368053:47) WARNING: start to do evaluation...
2023-05-06 07:06:16,302 (2306368053:60) WARNING: ema_eval_loss: 0.5590709447860718, eval_acc: 0.87768
2023-05-06 07:06:16,694 (2306368053:32) WARNING: epoch_index: 9, batch_index: 100, ema_loss: 0.20707310736179352
2023-05-06 07:06:17,075 (2306368053:32) WARNING: epoch_index: 9, batch_index: 120, ema_loss: 0.1997375637292862
2023-05-06 07:06:17,445 (2306368053:32

epoch: 100%|██████████| 10/10 [02:46<00:00, 16.63s/it]
